# Configuration

In [1]:
from ast import literal_eval
import joblib

import dotenv
dotenv.load_dotenv()

import pandas as pd

from jinja2 import Template
from typing import Dict, Any
from gemini_llm import GeminiLLM
from chroma_client import ChromaDBManager

from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper
from langchain_community.embeddings import SentenceTransformerEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_core.documents import Document

from ragas.testset import TestsetGenerator
from ragas import evaluate, RunConfig
from ragas.metrics import LLMContextRecall, LLMContextPrecisionWithReference, LLMContextPrecisionWithoutReference, AnswerRelevancy, AnswerCorrectness, AnswerSimilarity, Faithfulness
from datasets import Dataset  

In [2]:
PROMPT_TEMPLATE = """
{# Main story #}
Story:
{{ story }}

{# Search results #}
Relevant text excerpts:
{{ search_results }}

{# User question #}
Question: {{ user_question }}
"""

In [3]:
class RagTester:
    def __init__(self):
        self.chroma_manager = ChromaDBManager()
        self.llm = GeminiLLM()
        self.template = Template(PROMPT_TEMPLATE)
        self.chat_history = []

    def create_prompt(self, collection_name: str, query: str) -> str:
        """
        Создает промпт на основе истории, результатов поиска и вопроса пользователя
        """
        story = self.chroma_manager.get_complete_story(collection_name)
        search_results = self.chroma_manager.search(collection_name, query)

        prompt = self.template.render(
            story=story,
            search_results=search_results,
            user_question=query
        )

        return search_results, prompt

    def get_response(self, collection_name: str, query: str) -> Dict[str, Any]:
        """
        Получает ответ от модели с учетом контекста коллекции
        """
        search_results, full_prompt = self.create_prompt(
            collection_name=collection_name,
            query=query
        )
        
        response = self.llm.generate_response(
            full_prompt,
            chat_history=self.chat_history
        )
        
        # Обновляем историю чата
        self.chat_history.extend([
            {"role": "user", "parts": [{"text": query}]},
            {"role": "assistant", "parts": [{"text": response["text"]}]}
        ])
        
        return search_results, response

In [4]:
tester = RagTester()

<All keys matched successfully>


In [5]:
collections = tester.chroma_manager.get_base_collection_names()
print("Available collections:", collections)

INFO:chroma_client:Найдено 1 базовых коллекций: ['Sherlock_Study_in_Scarlet']


Available collections: ['Sherlock_Study_in_Scarlet']


In [6]:
collection_name = "Sherlock_Study_in_Scarlet"  # Укажите имя вашей коллекции Укажите ваш вопрос

# Ragas

## Generate

In [7]:
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
# generator_llm = LangchainLLMWrapper(llm)
# generator_embeddings = LangchainEmbeddingsWrapper(embedding_model)
model_name = "nomic-ai/nomic-embed-text-v1.5"
embedding_model = SentenceTransformerEmbeddings(model_name=model_name, model_kwargs={"trust_remote_code":True, 'device': 'cuda'})
generator_embeddings = LangchainEmbeddingsWrapper(embedding_model)

/tmp/ipykernel_4374/3403442604.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name=model_name, model_kwargs={"trust_remote_code":True, 'device': 'cuda'})
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: nomic-ai/nomic-embed-text-v1.5


In [17]:
testsetgenerator = TestsetGenerator(generator_llm, embedding_model=generator_embeddings)

In [12]:
# testsetgenerator.knowledge_graph = joblib.load('testsetgenerator_kg.pkl')
# testsetgenerator.persona_list = joblib.load('testsetgenerator_pl.pkl')

In [12]:
testsetgenerator

TestsetGenerator(llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), embedding_model=LangchainEmbeddingsWrapper(embeddings=HuggingFaceEmbeddings(...)), knowledge_graph=KnowledgeGraph(nodes: 0, relationships: 0), persona_list=None)

In [30]:
docs = []

for doc in tester.chroma_manager.client.get_collection(collection_name + '_chunks').peek()['documents']:
    docs.append(Document(metadata={"file_name": collection_name},
                         page_content=doc))

In [34]:
test_size = 100
testset = testsetgenerator.generate_with_langchain_docs(docs, testset_size=test_size)

Applying HeadlinesExtractor:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Applying HeadlineSplitter:   0%|          | 0/10 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/19 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:ragas.testset.synthesizers.multi_hop.abstract:found 112 clusters
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Requ

Generating Samples:   0%|          | 0/75 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

In [35]:
testset_pd = testset.to_pandas()
testset_pd.head()

,user_input,reference_contexts,reference,synthesizer_name
0,What kind of knowledge SHERLOCK HOLMES have?,[I was on the point of asking him what that wo...,SHERLOCK HOLMES has no knowledge of Literature...,single_hop_specifc_query_synthesizer
1,What kind of knowledge does SHERLOCK HOLMES po...,[I was on the point of asking him what that wo...,"According to the observations made, SHERLOCK H...",single_hop_specifc_query_synthesizer
2,In what ways does the character SHERLOCK HOLME...,[I was on the point of asking him what that wo...,The character SHERLOCK HOLMES demonstrates a s...,single_hop_specifc_query_synthesizer
3,What are the limits of SHERLOCK HOLMES's knowl...,[I was on the point of asking him what that wo...,"According to the context, SHERLOCK HOLMES has ...",single_hop_specifc_query_synthesizer
4,What are the limitations of SHERLOCK HOLMES's ...,[I was on the point of asking him what that wo...,"According to the context, SHERLOCK HOLMES has ...",single_hop_specifc_query_synthesizer


In [36]:
testset_pd.to_csv('synthetic_data_ragas_4o-mini.csv', index=False)

In [40]:
# joblib.dump(testsetgenerator.knowledge_graph, 'testsetgenerator_kg_llama_4o-mini.pkl')

['testsetgenerator_kg_llama_4o-mini.pkl']

In [41]:
# joblib.dump(testsetgenerator.persona_list, 'testsetgenerator_pl_llama_4o-mini.pkl') 

['testsetgenerator_pl_llama_4o-mini.pkl']

## Eval

In [44]:
testset_pd = pd.read_csv('synthetic_data_ragas_4o-mini.csv', converters={'reference_contexts': literal_eval})

In [45]:
testset_pd['reference_contexts']

0     [I was on the point of asking him what that wo...
1     [I was on the point of asking him what that wo...
2     [I was on the point of asking him what that wo...
3     [I was on the point of asking him what that wo...
4     [I was on the point of asking him what that wo...
                            ...                        
70    [<1-hop>\n\nI was on the point of asking him w...
71    [<1-hop>\n\nI was on the point of asking him w...
72    [<1-hop>\n\nI was on the point of asking him w...
73    [<1-hop>\n\nI was on the point of asking him w...
74    [<1-hop>\n\nI was on the point of asking him w...
Name: reference_contexts, Length: 75, dtype: object

In [46]:
testset_pd

,user_input,reference_contexts,reference,synthesizer_name
0,What kind of knowledge SHERLOCK HOLMES have?,[I was on the point of asking him what that wo...,SHERLOCK HOLMES has no knowledge of Literature...,single_hop_specifc_query_synthesizer
1,What kind of knowledge does SHERLOCK HOLMES po...,[I was on the point of asking him what that wo...,"According to the observations made, SHERLOCK H...",single_hop_specifc_query_synthesizer
2,In what ways does the character SHERLOCK HOLME...,[I was on the point of asking him what that wo...,The character SHERLOCK HOLMES demonstrates a s...,single_hop_specifc_query_synthesizer
3,What are the limits of SHERLOCK HOLMES's knowl...,[I was on the point of asking him what that wo...,"According to the context, SHERLOCK HOLMES has ...",single_hop_specifc_query_synthesizer
4,What are the limitations of SHERLOCK HOLMES's ...,[I was on the point of asking him what that wo...,"According to the context, SHERLOCK HOLMES has ...",single_hop_specifc_query_synthesizer
...,...,...,...,...
70,What can be inferred about Sherlock Holmes's k...,[<1-hop>\n\nI was on the point of asking him w...,Sherlock Holmes's knowledge in politics is des...,multi_hop_abstract_query_synthesizer
71,What insights can be drawn from Sherlock Holme...,[<1-hop>\n\nI was on the point of asking him w...,Sherlock Holmes' knowledge in politics is desc...,multi_hop_abstract_query_synthesizer
72,What insights can be drawn from Sherlock Holme...,[<1-hop>\n\nI was on the point of asking him w...,Sherlock Holmes' knowledge in politics is desc...,multi_hop_abstract_query_synthesizer
73,What can be inferred about Sherlock Holmes's k...,[<1-hop>\n\nI was on the point of asking him w...,Sherlock Holmes's knowledge in politics is des...,multi_hop_abstract_query_synthesizer


In [59]:
evalset_pd = pd.DataFrame({'user_input':testset_pd['user_input'],
                           'retrieved_contexts': [''] * len(testset_pd),
                           'reference_contexts': testset_pd['reference_contexts'],
                           'response': [''] * len(testset_pd),
                           'reference': testset_pd['reference']})

In [60]:
evalset_pd[['retrieved_contexts', 'response']] = evalset_pd.apply(lambda row: tester.get_response(collection_name, row['user_input']), axis=1, result_type ='expand')

INFO:chroma_client:Собрано 64 сцен в единый текст
INFO:chroma_client:Поиск по запросу: What kind of knowledge SHERLOCK HOLMES have?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:chroma_client:Собрано 64 сцен в единый текст
INFO:chroma_client:Поиск по запросу: What kind of knowledge does SHERLOCK HOLMES possess according to the observations made?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:chroma_client:Собрано 64 сцен в единый текст
INFO:chroma_client:Поиск по запросу: In what ways does the character SHERLOCK HOLMES demonstrate a selective approach to knowledge acquisition, as inferred from the context provided?


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:chroma_client:Собрано 64 сцен в единый текст
INFO:chroma_client:Поиск по запросу: What can be inferred about Sherlock Holmes's knowledge in politics and how does it relate to his overall inquiry skills?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:chroma_client:Собрано 64 сцен в единый текст
INFO:chroma_client:Поиск по запросу: What can be inferred about Sherlock Holmes's knowledge in politics and how it relates to his inquiry skills?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [61]:
evalset_pd.to_csv('synthetic_full_data_ragas_4o-mini_gemini.csv', index=False)

In [8]:
evalset_pd = pd.read_csv('synthetic_full_data_ragas_4o-mini_gemini.csv', converters={'retrieved_contexts': literal_eval, 'reference_contexts': literal_eval})

In [9]:
def parsing_context(context):
    
    retrieved_context = []
    
    for item_scene in context:
        if item_scene.get('chunks'):
            for chunk in item_scene['chunks']:
                retrieved_context.append(chunk['document'])
    
    return retrieved_context
    

In [10]:
evalset_pd['retrieved_contexts'] = evalset_pd.apply(lambda row: parsing_context(row['retrieved_contexts']), axis=1)

In [11]:
evalset_pd

,user_input,retrieved_contexts,reference_contexts,response,reference
0,What kind of knowledge SHERLOCK HOLMES have?,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': ""Based on the narrator's list, Sherlo...",SHERLOCK HOLMES has no knowledge of Literature...
1,What kind of knowledge does SHERLOCK HOLMES po...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': ""Based on the narrator's observations...","According to the observations made, SHERLOCK H..."
2,In what ways does the character SHERLOCK HOLME...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...",The character SHERLOCK HOLMES demonstrates a s...
3,What are the limits of SHERLOCK HOLMES's knowl...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': ""Based on the text provided, the limi...","According to the context, SHERLOCK HOLMES has ..."
4,What are the limitations of SHERLOCK HOLMES's ...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...","According to the context, SHERLOCK HOLMES has ..."
...,...,...,...,...,...
70,What can be inferred about Sherlock Holmes's k...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Okay, let\'s analyze what can be inf...",Sherlock Holmes's knowledge in politics is des...
71,What insights can be drawn from Sherlock Holme...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Okay, let\'s analyze Sherlock Holmes...",Sherlock Holmes' knowledge in politics is desc...
72,What insights can be drawn from Sherlock Holme...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Okay, let\'s analyze what can be inf...",Sherlock Holmes' knowledge in politics is desc...
73,What can be inferred about Sherlock Holmes's k...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Okay, let\'s analyze what can be inf...",Sherlock Holmes's knowledge in politics is des...


In [12]:
run_config = RunConfig(timeout=120, max_wait = 180, max_workers= 1)

In [13]:
score_result = evaluate(
  dataset=Dataset.from_pandas(evalset_pd),
  metrics=[
      LLMContextRecall(llm=generator_llm),
      LLMContextPrecisionWithReference(llm=generator_llm),
      LLMContextPrecisionWithoutReference(llm=generator_llm),
      AnswerCorrectness(llm=generator_llm, embeddings=generator_embeddings),
      AnswerRelevancy(llm=generator_llm, embeddings=generator_embeddings),
      AnswerSimilarity(embeddings=generator_embeddings),
      Faithfulness(llm=generator_llm)
  ]
)

Evaluating:   0%|          | 0/525 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[5]: TypeError('NoneType' object is not subscriptable)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1

In [14]:
score_result

{'context_recall': 0.6844, 'llm_context_precision_with_reference': 0.7368, 'llm_context_precision_without_reference': 0.7822, 'answer_correctness': 0.4175, 'answer_relevancy': 0.8714, 'semantic_similarity': 0.8031, 'faithfulness': 0.7847}

In [15]:
scoreset_pd = score_result.to_pandas()

In [16]:
scoreset_pd.to_csv('synthetic_score_data_ragas_4o-mini_gemini.csv', index=False)

In [39]:
scoreset_pd = pd.read_csv('synthetic_score_data_ragas_4o-mini_gemini.csv', converters={'retrieved_contexts': literal_eval, 'reference_contexts': literal_eval})

In [40]:
scoreset_pd

,user_input,retrieved_contexts,reference_contexts,response,reference,context_recall,llm_context_precision_with_reference,llm_context_precision_without_reference,answer_correctness,answer_relevancy,semantic_similarity,faithfulness
0,What kind of knowledge SHERLOCK HOLMES have?,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': ""Based on the narrator's list, Sherlo...",SHERLOCK HOLMES has no knowledge of Literature...,1.000000,1.000000,1.000000,0.187640,0.928529,NaN,1.000000
1,What kind of knowledge does SHERLOCK HOLMES po...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': ""Based on the narrator's observations...","According to the observations made, SHERLOCK H...",1.000000,1.000000,1.000000,0.371369,0.915705,0.735477,1.000000
2,In what ways does the character SHERLOCK HOLME...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...",The character SHERLOCK HOLMES demonstrates a s...,1.000000,1.000000,1.000000,0.450303,0.873316,0.841211,1.000000
3,What are the limits of SHERLOCK HOLMES's knowl...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': ""Based on the text provided, the limi...","According to the context, SHERLOCK HOLMES has ...",1.000000,1.000000,1.000000,0.465205,0.948736,0.801996,1.000000
4,What are the limitations of SHERLOCK HOLMES's ...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...","According to the context, SHERLOCK HOLMES has ...",1.000000,1.000000,1.000000,0.467892,0.930107,0.871568,0.785714
...,...,...,...,...,...,...,...,...,...,...,...,...
70,What can be inferred about Sherlock Holmes's k...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Okay, let\'s analyze what can be inf...",Sherlock Holmes's knowledge in politics is des...,0.666667,NaN,NaN,0.296683,0.989672,0.843876,0.951613
71,What insights can be drawn from Sherlock Holme...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Okay, let\'s analyze Sherlock Holmes...",Sherlock Holmes' knowledge in politics is desc...,0.333333,1.000000,0.805556,0.509576,0.908528,0.838305,0.803279
72,What insights can be drawn from Sherlock Holme...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Okay, let\'s analyze what can be inf...",Sherlock Holmes' knowledge in politics is desc...,0.666667,0.833333,1.000000,0.481113,0.962242,0.833542,0.903226
73,What can be inferred about Sherlock Holmes's k...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Okay, let\'s analyze what can be inf...",Sherlock Holmes's knowledge in politics is des...,0.666667,0.833333,1.000000,0.447299,0.987936,0.841827,0.754098


In [41]:
unique_scoreset_pd = scoreset_pd.iloc[scoreset_pd.user_input.drop_duplicates().index].copy()

In [43]:
unique_scoreset_pd

,user_input,retrieved_contexts,reference_contexts,response,reference,context_recall,llm_context_precision_with_reference,llm_context_precision_without_reference,answer_correctness,answer_relevancy,semantic_similarity,faithfulness
0,What kind of knowledge SHERLOCK HOLMES have?,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': ""Based on the narrator's list, Sherlo...",SHERLOCK HOLMES has no knowledge of Literature...,1.000000,1.000000,1.000000,0.187640,0.928529,NaN,1.000000
1,What kind of knowledge does SHERLOCK HOLMES po...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': ""Based on the narrator's observations...","According to the observations made, SHERLOCK H...",1.000000,1.000000,1.000000,0.371369,0.915705,0.735477,1.000000
2,In what ways does the character SHERLOCK HOLME...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...",The character SHERLOCK HOLMES demonstrates a s...,1.000000,1.000000,1.000000,0.450303,0.873316,0.841211,1.000000
3,What are the limits of SHERLOCK HOLMES's knowl...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': ""Based on the text provided, the limi...","According to the context, SHERLOCK HOLMES has ...",1.000000,1.000000,1.000000,0.465205,0.948736,0.801996,1.000000
4,What are the limitations of SHERLOCK HOLMES's ...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...","According to the context, SHERLOCK HOLMES has ...",1.000000,1.000000,1.000000,0.467892,0.930107,0.871568,0.785714
5,What are the limitations of SHERLOCK HOLMES's ...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...","According to the context, SHERLOCK HOLMES has ...",1.000000,1.000000,1.000000,0.519048,0.922658,0.876192,1.000000
6,What does the document reveal about SHERLOCK H...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,{'text': 'The document reveals the following a...,The document reveals that SHERLOCK HOLMES poss...,1.000000,NaN,NaN,0.340213,0.952657,0.831440,0.935484
7,Wot are the limits of SHERLOCK HOLMES's knowle...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...",The limits of SHERLOCK HOLMES's knowledge incl...,1.000000,1.000000,1.000000,0.419258,0.936305,0.819888,1.000000
8,What SHERLOCK HOLMES know?,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...",SHERLOCK HOLMES has no knowledge of Literature...,1.000000,1.000000,NaN,0.291871,0.834985,0.792485,1.000000
9,What are the limits of SHERLOCK HOLMES's knowl...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...","The limits of SHERLOCK HOLMES's knowledge, as ...",1.000000,1.000000,1.000000,0.560403,0.937915,0.856995,1.000000


In [44]:
unique_scoreset_pd.iloc[:, 5:].mean()

context_recall                             0.682099
llm_context_precision_with_reference       0.718143
llm_context_precision_without_reference    0.722334
answer_correctness                         0.419754
answer_relevancy                           0.840427
semantic_similarity                        0.792096
faithfulness                               0.827878
dtype: float64

In [18]:
filter_scoreset_pd = scoreset_pd[(scoreset_pd.iloc[:, 5:] > 0.5).mean(axis=1) == 1].reset_index(drop=True).copy()

In [29]:
filter_scoreset_pd

,user_input,retrieved_contexts,reference_contexts,response,reference,context_recall,llm_context_precision_with_reference,llm_context_precision_without_reference,answer_correctness,answer_relevancy,semantic_similarity,faithfulness
0,What are the limitations of SHERLOCK HOLMES's ...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...","According to the context, SHERLOCK HOLMES has ...",1.000000,1.000000,1.000000,0.519048,0.922658,0.876192,1.000000
1,What are the limits of SHERLOCK HOLMES's knowl...,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...","The limits of SHERLOCK HOLMES's knowledge, as ...",1.000000,1.000000,1.000000,0.560403,0.937915,0.856995,1.000000
2,Wot is the limitashuns of SHERLOCK HOLMES?,[I was on the point of asking him what that wo...,[I was on the point of asking him what that wo...,"{'text': 'Based on the provided text excerpts,...",The limits of SHERLOCK HOLMES include a lack o...,1.000000,1.000000,1.000000,0.904740,0.780584,0.849729,1.000000
3,What knowledge does the character possess abou...,[I was on the point of asking him what that wo...,[Politics.—Feeble. 5. Botany.—Variable. Well u...,"{'text': 'Based on the text provided, Sherlock...",The character has a good practical knowledge o...,1.000000,0.805556,1.000000,0.561145,0.817259,0.744579,1.000000
4,What are some of the diverse skills and knowle...,[I was on the point of asking him what that wo...,[Politics.—Feeble. 5. Botany.—Variable. Well u...,"{'text': 'Based on the provided text, here are...",Sherlock Holmes possesses a wide range of skil...,1.000000,0.833333,0.833333,0.522641,0.940241,0.863290,1.000000
5,What can you tell me about the different types...,[I was on the point of asking him what that wo...,[Politics.—Feeble. 5. Botany.—Variable. Well u...,"{'text': 'Based on the provided text excerpts,...",Sherlock Holmes interacts with a variety of in...,1.000000,0.826020,0.780159,0.561652,0.857832,0.746610,0.947368
6,What insights can be drawn from Sherlock Holme...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Based on the provided text, Sherlock...",Sherlock Holmes' knowledge in politics is desc...,0.666667,1.000000,0.805556,0.517745,0.906056,0.820978,0.916667
7,In what ways does Sherlock Holmes's knowledge ...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Okay, let\'s analyze how Sherlock Ho...","Sherlock Holmes's knowledge in various fields,...",0.750000,1.000000,0.810000,0.645072,0.932181,0.802511,0.971014
8,In what ways does Sherlock Holmes's knowledge ...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Okay, let\'s analyze Sherlock Holmes...",Sherlock Holmes's knowledge of politics is des...,1.000000,1.000000,0.805556,0.539068,0.941295,0.822941,0.964912
9,What insights can be drawn from Sherlock Holme...,[I was on the point of asking him what that wo...,[<1-hop>\n\nI was on the point of asking him w...,"{'text': 'Based on the provided text excerpts,...",Sherlock Holmes' knowledge in politics is desc...,1.000000,1.000000,1.000000,0.510293,0.940419,0.841170,0.932203


In [19]:
print('Len good dataset:', len(filter_scoreset_pd))

Len good dataset: 11


In [30]:
scoreset_pd.iloc[:, 5:].mean()

context_recall                             0.684444
llm_context_precision_with_reference       0.736769
llm_context_precision_without_reference    0.782235
answer_correctness                         0.417457
answer_relevancy                           0.871395
semantic_similarity                        0.803132
faithfulness                               0.784748
dtype: float64

In [31]:
filter_scoreset_pd.iloc[:, 5:].mean()

context_recall                             0.924242
llm_context_precision_with_reference       0.951355
llm_context_precision_without_reference    0.912237
answer_correctness                         0.586212
answer_relevancy                           0.904799
semantic_similarity                        0.822828
faithfulness                               0.972155
dtype: float64

In [22]:
def print_score_pd(scoreset_pd, idx):
    print('user_input:', scoreset_pd.iloc[idx]['user_input'], '\n')
    print('retrieved_contexts:', scoreset_pd.iloc[idx]['retrieved_contexts'], '\n')
    print('reference_contexts:', scoreset_pd.iloc[idx]['reference_contexts'], '\n')
    print('response:', scoreset_pd.iloc[idx]['response'], '\n')
    print('reference:', scoreset_pd.iloc[idx]['reference'], '\n')
    
    print('context_recall =', scoreset_pd.iloc[idx]['context_recall'])
    print('llm_context_precision_with_reference =', scoreset_pd.iloc[idx]['llm_context_precision_with_reference'])
    print('llm_context_precision_without_reference =', scoreset_pd.iloc[idx]['llm_context_precision_without_reference'])
    print('answer_correctness =', scoreset_pd.iloc[idx]['answer_correctness'])
    print('answer_relevancy =', scoreset_pd.iloc[idx]['answer_relevancy'])
    print('semantic_similarity =', scoreset_pd.iloc[idx]['semantic_similarity'])
    print('faithfulness =', scoreset_pd.iloc[idx]['faithfulness'])

In [27]:
print_score_pd(scoreset_pd, 70)

user_input: What can be inferred about Sherlock Holmes's knowledge in politics and how it relates to his inquiry skills? 

retrieved_contexts: ['I was on the point of asking him what that work might be, but something\nin his manner showed me that the question would be an unwelcome one. I\npondered over our short conversation, however, and endeavoured to draw\nmy deductions from it. He said that he would acquire no knowledge which\ndid not bear upon his object. Therefore all the knowledge which he\npossessed was such as would be useful to him. I enumerated in my own\nmind all the various points upon which he had shown me that he was\nexceptionally well-informed. I even took a pencil and jotted them down.\nI could not help smiling at the document when I had completed it. It\nran in this way—\n\n\nSHERLOCK HOLMES—his limits.\n\n\n1. Knowledge of Literature.—Nil.\n2. Philosophy.—Nil.\n3. Astronomy.—Nil.\n4. Politics.—Feeble.\n5. Botany.—Variable. Well up in belladonna, opium, and poisons\n

In [28]:
print_score_pd(scoreset_pd, 5)

user_input: What are the limitations of SHERLOCK HOLMES's knowledge as inferred from the context? 

retrieved_contexts: ['I was on the point of asking him what that work might be, but something\nin his manner showed me that the question would be an unwelcome one. I\npondered over our short conversation, however, and endeavoured to draw\nmy deductions from it. He said that he would acquire no knowledge which\ndid not bear upon his object. Therefore all the knowledge which he\npossessed was such as would be useful to him. I enumerated in my own\nmind all the various points upon which he had shown me that he was\nexceptionally well-informed. I even took a pencil and jotted them down.\nI could not help smiling at the document when I had completed it. It\nran in this way—\n\n\nSHERLOCK HOLMES—his limits.\n\n\n1. Knowledge of Literature.—Nil.\n2. Philosophy.—Nil.\n3. Astronomy.—Nil.\n4. Politics.—Feeble.\n5. Botany.—Variable. Well up in belladonna, opium, and poisons\ngenerally. Knows nothin